# Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Basic libraries
#
import random
import time
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt

# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Sklearn library
#
from sklearn.preprocessing   import StandardScaler


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Tensorflow library
#
import tensorflow                as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks  import TerminateOnNaN, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils      import plot_model


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# User libraries
#
from utils.metrics      import PEHE, ATE
from utils.Loss         import *
from utils.DragonNet    import *
from utils.utils        import MSE
from utils.dataloader   import *

print('[INFO] All libraries were imported')

# Parameters

In [ ]:
Problem = 'Synthetic'
problem_type = 'linear'  # 'linear', 'sin'
train_size = 0.8
n_f, n_i = 10, 5
nInstances = 1000 # size
p = 0.3
seed = 42

Model = 'Smart-DragonNet'



random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
# tf.random.set_seed(seed) # for Tensoflow >= 2.0
tf.random.set_random_seed(seed)

In [ ]:
targeted_regularization    = True # {True, False}

output_dir                 = ''
knob_loss                  = dragonnet_loss_binarycross
ratio                      = 1.
validation_split           = 0.2
batch_size                 = 64
verbose                    = False




metrics = [regression_loss, binary_classification_loss, treatment_accuracy, track_epsilon]

if targeted_regularization:
    loss = make_tarreg_loss(ratio=ratio, dragonnet_loss=knob_loss)
else:
    loss = knob_loss

In [ ]:
DataLoader = Synthetic(type=problem_type, size=nInstances, n_f=n_f, n_i=n_i, p=p, seed=seed)
DataLoader.create_dataset(train_size=0.8)

# Simulations

In [ ]:
# Start timer
#
start1 = time.time()


# Load training data
#
trainX, trainT, trainY, train_potential_Y = DataLoader.get_training_data()

# Load testing data
#
testX, testT, testY, test_potential_Y = DataLoader.get_testing_data()
#
print('[INFO] Datasets imported')

# Remove irrelevant features
trainX = trainX[:, n_i:]
testX = testX[:, n_i:]

# Setup scaler for inputs
scalerX = StandardScaler()
trainX  = scalerX.fit_transform( trainX )
testX   = scalerX.transform( testX )


# Setup scaler for target variable
#
scalerY       = StandardScaler()
trainY_scaled = scalerY.fit_transform( trainY.reshape(-1,1) ).squeeze(-1)




# Setup DragonNet
#
dragonnet = make_dragonnet(trainX.shape[1], 0.01)


# Create outputs for DragonNet (concatenate Y & T)
#
yt_train = np.concatenate([trainY_scaled.reshape(-1,1), trainT.reshape(-1,1)], axis = 1)


#
#
# *** Training - Phase I ***
#
#

# Compile network
#
dragonnet.compile(optimizer = Adam(lr=1e-3), 
                    loss      = loss, 
                    metrics   = metrics)

# Setup callbacks
callbacks = [TerminateOnNaN(),
                EarlyStopping(monitor   = 'val_loss', 
                            patience  = 2, 
                            min_delta = 0.),
                ReduceLROnPlateau(monitor   = 'loss', 
                                factor    = 0.5, 
                                patience  = 5, 
                                verbose   = verbose, 
                                mode      = 'auto', 
                                min_delta = 1e-8, 
                                cooldown  = 0, 
                                min_lr    = 0)]

start_time = time.time()

# Training
#
dragonnet.fit(trainX, yt_train, 
                callbacks        = callbacks,
                validation_split = validation_split,
                epochs           = 100,
                batch_size       = batch_size, 
                verbose          = verbose)


print("[INFO] Training - Phase I - Time %.2f secs" % (time.time() - start_time) )







#
#
# *** Training - Phase II ***
#
#    

# Setup callbacks
#
callbacks = [TerminateOnNaN(),
                EarlyStopping(monitor   = 'val_loss', 
                            patience  = 40, 
                            min_delta = 0.),
                ReduceLROnPlateau(monitor   = 'loss', 
                                factor    = 0.5, 
                                patience  = 5, 
                                verbose   = verbose, 
                                mode      = 'auto',
                                min_delta = 0., 
                                cooldown  = 0, 
                                min_lr    = 0)
]

# Compile network
#
dragonnet.compile(optimizer = SGD(lr=1e-5, momentum=0.9, nesterov=True), 
                    loss      = loss,
                    metrics   = metrics)




start_time = time.time()

# Training
#
dragonnet.fit(trainX, yt_train, 
                callbacks        = callbacks,
                validation_split = validation_split,
                epochs           = 300,
                batch_size       = batch_size, 
                verbose          = verbose)

print("[INFO] Training - Phase II - Time %.2f secs" % (time.time() - start_time) )





#
#
# *** Predictions ***
#
#       
yt_hat_test  = dragonnet.predict( testX )

# Get predictions
#
test_y_hat = yt_hat_test[:,:2]

# Apply inverse transformation
#
test_y_hat[:,0] = scalerY.inverse_transform( test_y_hat[:,0].reshape(-1,1) )[0]
test_y_hat[:,1] = scalerY.inverse_transform( test_y_hat[:,1].reshape(-1,1) )[0]

# Get propensity score
#
propensity_score = yt_hat_test[:,2]




# Results
Results = dict()
Results['Error_PEHE'] = PEHE(test_potential_Y, test_y_hat)
Results['Error_ATE'] = ATE(test_potential_Y, test_y_hat)  
Results['MSE_0'] = MSE(test_potential_Y[:,0], test_y_hat[:,0])
Results['MSE_1'] = MSE(test_potential_Y[:,1], test_y_hat[:,1])

    
print('[INFO] Error of PEHE and ATE computed')
print('[INFO] Time %.2f\n\n' % (time.time() - start1))

In [ ]:
import json
filename = f'Results/Problem={Problem}-{problem_type}-{p}_Model={Model}.json'

with open(filename, "w") as outfile:
    json.dump(Results, outfile)

Results

# {'Error_PEHE': 1772.8156294658145,
#  'Error_ATE': 41.191034161971594,
#  'MSE_0': 322100.54965537897,
#  'MSE_1': 281469.1645955287}

# Feature Importance

## Permutation Feature Importance

In [ ]:
# Set feature names
Features = [f'X{i+1}' for i in range(testX.shape[1])]

# Number of Iteration for Permutation-Feature-Importance
nSimulations = 100

In [ ]:
from tqdm import tqdm

Performance = {'Error_PEHE': [], 'Error_ATE': [], 'MSE_0': [], 'MSE_1':  [], 'Feature': [], 'Iteration': []}

# Features
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
for i in range(testX.shape[1]):
    print('Feature: ', Features[i])

    for Sim in tqdm(range(nSimulations)):
        testX_new = testX.copy()
        
        # Shuffle Feature-i
        np.random.shuffle(testX_new[:,0])

        # Get predictions
        yt_hat_test  = dragonnet.predict( testX_new )
        test_y_hat = yt_hat_test[:,:2]

        # Apply inverse transformation
        test_y_hat[:,0] = scalerY.inverse_transform( test_y_hat[:,0].reshape(-1,1) )[0]
        test_y_hat[:,1] = scalerY.inverse_transform( test_y_hat[:,1].reshape(-1,1) )[0]

        # Include Feature performance
        Performance['Error_PEHE'] += [PEHE(test_potential_Y, test_y_hat)]
        Performance['Error_ATE'] += [ATE(test_potential_Y, test_y_hat)  ]
        Performance['MSE_0'] += [MSE(test_potential_Y[:,0], test_y_hat[:,0])]
        Performance['MSE_1'] += [MSE(test_potential_Y[:,1], test_y_hat[:,1])]
        Performance['Feature'] += [Features[i]]
        Performance['Iteration'] += [Sim]

In [ ]:
df = pd.DataFrame.from_dict(Performance)

for performanceMetric in ['Error_PEHE', 'Error_ATE', 'MSE_0', 'MSE_1']:
    df[performanceMetric] = df[performanceMetric] - Results[performanceMetric]


df.to_csv(f'PFI/Problem={Problem}-{problem_type}-{p}_Model={Model}.csv', index = False)

## Shap

In [ ]:
import shap
explainer = shap.DeepExplainer(dragonnet, trainX)

shap_values = explainer.shap_values(testX)

np.savez(f'SHAP/Problem={Problem}-{problem_type}-{p}_Model={Model}.npz', shap_values=shap_values, Features=np.array(Features))
